<a href="https://colab.research.google.com/github/Kryptera-K/MU-Weighted-Pullback-Trend-Filter-Strategy/blob/main/MU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install vectorbt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.7/527.7 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 46.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
import vectorbt as vbt

# -------------------------
# Download Data
# -------------------------

symbol = "MU"
start_date = "2000-01-01"
end_date = "2026-01-01"
interval = "1d"

df = yf.download(symbol, start=start_date, end=end_date, interval=interval, multi_level_index=False)
df.to_csv("MU_clean.csv", index=False)
df

/tmp/ipython-input-3573611977.py:15: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(symbol, start=start_date, end=end_date, interval=interval, multi_level_index=False)
[*********************100%***********************]  1 of 1 completed


,Close,High,Low,Open,Volume
Date,,,,,
2000-01-03,37.060966,38.128566,35.383310,37.762532,6564600
2000-01-04,35.627316,36.999943,35.566310,36.481395,7220800
2000-01-05,35.505306,36.481397,34.193684,35.200278,6763800
2000-01-06,36.115372,36.755932,35.017269,35.505315,6338400
2000-01-07,34.407204,35.993351,33.187090,35.993351,9467200
...,...,...,...,...,...
2025-10-14,187.059998,192.460007,186.250000,186.960007,15912400
2025-10-15,191.940002,192.880005,187.419998,192.479996,17793100
2025-10-16,202.529999,206.339996,195.199997,199.960007,42041300


In [ ]:
# -------------------------
# Necessary Parameters
# -------------------------

EMA_PERIOD = 20
EMA_SHIFT = 5
VOLUME_SHIFT = 10
WMA_PERIOD = 20
WMA_SHIFT = 5
WOODIE_CCI_PERIOD = 14
WOODIE_FAST_PERIOD = 6
WOODIE_SHIFT = 5
WOODIE_SLOW_PERIOD = 34

# -------------------------
# Indicator Functions
# -------------------------

def open_below_wma_after_above(df, period=WMA_PERIOD):
    df = calculate_wma(df, period)
    return (df['Open'] < df['WMA']) & (df['Open'].shift(1) > df['WMA'].shift(1))


def calculate_wma(df, period=WMA_PERIOD):
    """
    Calculate Weighted Moving Average (WMA) for the Close price
    and add it as a column to the DataFrame.
    """
    df = df.copy()
    weights = np.arange(1, period + 1)
    df['WMA'] = df['Close'].rolling(period).apply(lambda prices: np.dot(prices, weights) / weights.sum(), raw=True)
    return df


def volume_is_falling(df, shift=VOLUME_SHIFT):
    return df['Volume'] < df['Volume'].shift(shift)


def ema_rising(df, shift=EMA_SHIFT, period=EMA_PERIOD):
    df = calculate_ema(df, period)
    return df['EMA'] > df['EMA'].shift(shift)


def calculate_ema(df, period=EMA_PERIOD):
    """
    Calculate Exponential Moving Average (EMA) of the Close price.
    """
    df = df.copy()
    df['EMA'] = df['Close'].ewm(span=period, adjust=False).mean()
    return df


def woodies_trend_down(df, period=WOODIE_SLOW_PERIOD):
    df['CCI_trend'] = calculate_woodie_cci(df, period)['CCI']
    return df['CCI_trend'] < 0


def calculate_woodie_cci(df, period=WOODIE_CCI_PERIOD):
    """
    Calculate Commodity Channel Index (CCI).
    """
    df = df.copy()
    # Typical Price
    df['TP'] = (df['High'] + df['Low'] + df['Close']) / 3
    # SMA of TP
    df['SMA_TP'] = df['TP'].rolling(period).mean()
    # Mean Deviation
    df['MeanDev'] = df['TP'].rolling(period).apply(
        lambda x: np.mean(np.abs(x - np.mean(x))), raw=True
    )
    # CCI
    df['CCI'] = (df['TP'] - df['SMA_TP']) / (0.015 * df['MeanDev'])
    return df



# -------------------------
# Entry conditions
# -------------------------

df["WMA_Open_Below_After_Above"] = open_below_wma_after_above(df)
df["Volume_Falling"] = volume_is_falling(df)

# -------------------------
# Exit conditions
# -------------------------

df["EMA_Rising"] = ema_rising(df)
df["Woodies_Trend_Down"] = woodies_trend_down(df)

# -------------------------
# Signals
# -------------------------

entry_conditions = [
    'WMA_Open_Below_After_Above',
    'Volume_Falling',
]
exit_conditions = [
    'EMA_Rising',
    'Woodies_Trend_Down',
]

df['entry_signal'] = df[entry_conditions].all(axis=1)
df['exit_signal']  = df[exit_conditions].all(axis=1)

# -------------------------
# Backtest
# -------------------------


shift_entries = df['entry_signal'].shift(1).astype(bool).fillna(False).to_numpy()
shift_exits = df['exit_signal'].shift(1).astype(bool).fillna(False).to_numpy()

pf = vbt.Portfolio.from_signals(
    close=df['Open'],
    entries=shift_entries,
    exits=shift_exits,
    init_cash=100_000,
    fees=0.001,
    slippage=0.002,
    freq='1d'
)


# -------------------------
# Portfolio Stats / Plot
# -------------------------

print(pf.stats())
pf.plot().show()

Start                                2000-01-03 00:00:00
End                                  2025-10-20 00:00:00
Period                                6489 days 00:00:00
Start Value                                     100000.0
End Value                                 2834149.724275
Total Return [%]                             2734.149724
Benchmark Return [%]                          455.193169
Max Gross Exposure [%]                             100.0
Total Fees Paid                            133254.178762
Max Drawdown [%]                                62.56007
Max Drawdown Duration                 1446 days 00:00:00
Total Trades                                         110
Total Closed Trades                                  109
Total Open Trades                                      1
Open Trade PnL                            1239962.117001
Win Rate [%]                                   49.541284
Best Trade [%]                                 94.674705
Worst Trade [%]                

In [ ]:
# Buy and Hold Performance Metrics
df_holding = df['Open']
pf_holding = vbt.Portfolio.from_holding(df_holding, init_cash=100_000 , freq='D')
print(pf_holding.stats())

Start                         2000-01-03 00:00:00
End                           2025-10-20 00:00:00
Period                         6489 days 00:00:00
Start Value                              100000.0
End Value                            555193.16913
Total Return [%]                       455.193169
Benchmark Return [%]                   455.193169
Max Gross Exposure [%]                      100.0
Total Fees Paid                               0.0
Max Drawdown [%]                        98.195876
Max Drawdown Duration          5943 days 00:00:00
Total Trades                                    1
Total Closed Trades                             0
Total Open Trades                               1
Open Trade PnL                       455193.16913
Win Rate [%]                                  NaN
Best Trade [%]                                NaN
Worst Trade [%]                               NaN
Avg Winning Trade [%]                         NaN
Avg Losing Trade [%]                          NaN


In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# -------------------------
# Download Data
# -------------------------
symbol = "MU"
start_date = "2000-01-01"
end_date = "2026-01-01"
interval = "1d"

df = yf.download(symbol, start=start_date, end=end_date, interval=interval, multi_level_index=False)

# -------------------------
# Parameters
# -------------------------
EMA_PERIOD = 20
WMA_PERIOD = 20
WOODIE_CCI_PERIOD = 14

# -------------------------
# Indicator Functions
# -------------------------
def calculate_wma(df, period=WMA_PERIOD):
    weights = np.arange(1, period + 1)
    df['WMA'] = df['Close'].rolling(period).apply(lambda prices: np.dot(prices, weights) / weights.sum(), raw=True)
    return df

def calculate_ema(df, period=EMA_PERIOD):
    df['EMA'] = df['Close'].ewm(span=period, adjust=False).mean()
    return df

def calculate_woodie_cci(df, period=WOODIE_CCI_PERIOD):
    df['TP'] = (df['High'] + df['Low'] + df['Close']) / 3
    df['SMA_TP'] = df['TP'].rolling(period).mean()
    df['MeanDev'] = df['TP'].rolling(period).apply(lambda x: np.mean(np.abs(x - np.mean(x))), raw=True)
    df['CCI'] = (df['TP'] - df['SMA_TP']) / (0.015 * df['MeanDev'])
    return df

# -------------------------
# Compute Indicators
# -------------------------
df = calculate_ema(df)
df = calculate_wma(df)
df = calculate_woodie_cci(df)
df.dropna(inplace=True)

# -------------------------
# Plot
# -------------------------
fig = make_subplots(
    rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.03,
    row_heights=[0.6, 0.25, 0.15],
    subplot_titles=(f"{symbol} Price with EMA & WMA", "Woodie CCI", "Volume")
)

# --- Candlestick + EMA + WMA ---
fig.add_trace(go.Candlestick(
    x=df.index, open=df['Open'], high=df['High'],
    low=df['Low'], close=df['Close'],
    name='Price', increasing_line_color='green', decreasing_line_color='red',
    showlegend=False
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=df.index, y=df['EMA'], mode='lines', line=dict(width=1.5, color='blue'),
    name=f'EMA ({EMA_PERIOD})'
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=df.index, y=df['WMA'], mode='lines', line=dict(width=1.5, color='orange'),
    name=f'WMA ({WMA_PERIOD})'
), row=1, col=1)

# --- Woodie CCI ---
fig.add_trace(go.Scatter(
    x=df.index, y=df['CCI'], mode='lines', line=dict(color='purple', width=1.5),
    name='Woodie CCI'
), row=2, col=1)

# Zero Line
fig.add_trace(go.Scatter(
    x=df.index, y=[0]*len(df), mode='lines', line=dict(color='gray', dash='dot'),
    name='Zero Line'
), row=2, col=1)

# --- Volume ---
fig.add_trace(go.Bar(
    x=df.index, y=df['Volume'], name='Volume', marker_color='green'
), row=3, col=1)

# -------------------------
# Layout Settings
# -------------------------
fig.update_layout(
    title=f"{symbol} Indicators Overview",
    xaxis_rangeslider_visible=False,
    template="plotly_white",
    height=900,
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
)

fig.update_yaxes(title_text="Price", row=1, col=1)
fig.update_yaxes(title_text="CCI", row=2, col=1)
fig.update_yaxes(title_text="Volume", row=3, col=1)

fig.show()


/tmp/ipython-input-2747964095.py:15: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  1 of 1 completed
